## Mod 2 Assessment

In [1]:
# put all imports here
import json
import requests
import pandas as pd
import pymongo
import sqlite3
from bs4 import BeautifulSoup
from bson.json_util import loads

### SQL (10 Minutes)
There is a sqlite3 database in `assets/books.db`. The SQL to create this is also in `assets/books.sql`, if you want to run it manually (you can also import this using https://sqliteonline.com/, or run the SQL file directly).  Both have the same schema and data.

The schema has three tables. You can explore the schema in the fiddle posted above.  Please answer the following questions.

1. Connect to the database using sqlite3

In [2]:
conn = sqlite3.connect('assets/books.db')
cur = conn.cursor()

2. Querying the DB: how many pages are in the book "Nine Stories"?

In [8]:
def prettify(data):
    db = pd.DataFrame(data)
    db.columns = [d[0] for d in cur.description]
    return db

In [10]:
cur.execute("""
    SELECT *
    FROM sqlite_master
    WHERE TYPE='table'
    """)
prettify(cur.fetchall())

,type,name,tbl_name,rootpage,sql
0,table,author,author,2,"CREATE TABLE author (\n author_id INT,\n ..."
1,table,book,book,3,"CREATE TABLE book (\n book_id INT,\n tit..."
2,table,book_author,book_author,4,"CREATE TABLE book_author (\n author_id INT,..."


In [18]:
cur.execute("""
    SELECT *
    FROM book
    """)
db = prettify(cur.fetchall())
db.head()

,book_id,title,pages
0,1,The Catcher in the Rye,500
1,2,Nine Stories,600
2,3,Franny and Zooey,700
3,4,The Great Gatsby,800
4,5,Tender id the Night,200


In [13]:
cur.execute("""
    SELECT *
    FROM book
    WHERE title like 'Nine Stories'
    """)
prettify(cur.fetchall())

,book_id,title,pages
0,2,Nine Stories,600


3. How many authors are from the USA?

In [15]:
cur.execute("""
    SELECT *
    FROM author
    """)
db = prettify(cur.fetchall())
db.head()

,author_id,name,country
0,1,J.D. Salinger,USA
1,2,F. Scott. Fitzgerald,USA
2,3,Jane Austen,UK
3,4,Scott Hanselman,USA
4,5,Jason N. Gaylord,USA


In [24]:
cur.execute("""
    SELECT country, COUNT(*)
    FROM author
    WHERE country like 'USA'
    """)
prettify(cur.fetchall())

,country,COUNT(*)
0,USA,6


4. How many authors does the book "Professional ASP.NET 4.5 in C# and VB" have?

In [19]:
cur.execute("""
    SELECT *
    FROM book_author
    """)
db = prettify(cur.fetchall())
db.head()

,author_id,book_id
0,1,1
1,1,2
2,1,3
3,2,4
4,2,5


In [25]:
cur.execute("""
    SELECT title, count(author_id) as num_authors
    FROM book
    JOIN book_author USING(book_id)
    WHERE title LIKE 'Professional ASP.NET 4.5 in C# and VB'
    """)
prettify(cur.fetchall())

,title,num_authors
0,Professional ASP.NET 4.5 in C# and VB,5


5. How many pages total have been written by non-American authors?

In [27]:
cur.execute("""
    SELECT SUM(pages) as total_pages
    FROM book
    JOIN book_author USING(book_id)
    JOIN author
    WHERE title NOT LIKE 'USA'
    """)
prettify(cur.fetchall())

,total_pages
0,262520


### Object Oriented Programming (15 Minutes)
Below is the stub of some classes. Make the classes adhere to the following rules. The code beneath the cell should  run successfully.
1. Force every new instance of `WeWorkMember` to pass in a value to set the attribute `name`.
1. Give every new instance of `WeWorkMember` an attribute `caffeinated` that is set to `False`. 
1. Give `WeWorkMember` an **instance** method called `caffeinate()` that prints out "Getting coffee!" and sets the attribute `caffeinated` to `True`.
1. Make it so that `Staff` and `Student` inherit from `WeWorkMember`
1. Give `Staff` a **static** method called `cheer()` that prints out "Goooooooo Flatiron Students!"
1. Give `Students` a **class** method called `learn()` that takes in an integer and returns that number +1

In [31]:
# Modify this cell
class WeWorkMember:
    def __init__(self, name):
        self.name = name
        
    caffeinated = False
    
    def caffeinate(self):
        print('Getting coffee!')
        self.caffeinated = True

class Staff(WeWorkMember):
    @staticmethod
    def cheer():
        print('Goooooooo Flatiron Students!')

class Student(WeWorkMember):
    @classmethod
    def learn(cls, num):
        return num + 1

    

In [32]:
# DO NOT MODIFY this cell. if you want to add more tests, add them in a new cell!
try:
    nobody = Staff()
    raise RuntimeError('Staff needs a name!')
except TypeError:
    pass

andy = Staff('Andy')
assert andy.caffeinated is False
andy.caffeinate()
assert andy.caffeinated is True

Staff.cheer()

bobby = Student('Bobby Tables')
assert bobby.caffeinated is False
bobby.caffeinate()
assert bobby.caffeinated is True

Student.learn(99)

Getting coffee!
Goooooooo Flatiron Students!
Getting coffee!


100

### APIs (10 Minutes)
Using this API (http://www.nokeynoshade.party/api/judges) about RuPaul's Drag Race (docs: https://drag-race-api.readme.io/docs/get-all-judges), tell me how many judges of each **`type`** there were in the first 50 records the API returns.
 - the 50 judges will have **`id`** ranging from 1 to 53
 - you can do the aggregation in pure Python, Pandas, SQL-- whatever's easiest for you

In [44]:
resp = requests.get('http://www.nokeynoshade.party/api/judges', params={'limit': 50})
resp.status_code

200

In [45]:
resp.json()

[{'id': 1,
  'name': 'Rupaul',
  'image_url': 'https://vignette.wikia.nocookie.net/logosrupaulsdragrace/images/b/ba/Rupaul_blackpink_final.jpg/revision/latest/scale-to-width-down/350?cb=20110731183922',
  'bio': 'Drag performer, actor, television host, and recording artist',
  'type': 'regular',
  'createdAt': '2018-01-11T00:43:31.258Z',
  'updatedAt': '2018-01-11T00:43:31.258Z'},
 {'id': 2,
  'name': 'Bob Mackie',
  'image_url': None,
  'bio': 'Fashion Designer',
  'type': 'guest',
  'createdAt': '2017-08-22T00:27:40.731Z',
  'updatedAt': '2017-08-22T00:27:40.731Z'},
 {'id': 3,
  'name': 'Mike Ruiz',
  'image_url': 'https://vignette3.wikia.nocookie.net/logosrupaulsdragrace/images/7/7c/Mike_Ruiz.jpg/revision/latest/scale-to-width-down/350?cb=20110731184513',
  'bio': 'Photographer',
  'type': 'regular',
  'createdAt': '2017-08-22T00:46:35.279Z',
  'updatedAt': '2017-08-22T00:46:35.279Z'},
 {'id': 4,
  'name': 'Michelle Williams',
  'image_url': 'https://upload.wikimedia.org/wikipedia/c

In [47]:
resp.json()[0].keys()

dict_keys(['id', 'name', 'image_url', 'bio', 'type', 'createdAt', 'updatedAt'])

In [53]:
count_of_judge_types = {'regular': 0, 'guest': 0, 'interim': 0}
for judge in range(50):
    if resp.json()[judge]['type'] == 'regular':
        count_of_judge_types['regular'] += 1
    if resp.json()[judge]['type'] == 'guest':
        count_of_judge_types['guest'] += 1
    if resp.json()[judge]['type'] == 'interim':
        count_of_judge_types['interim'] += 1
count_of_judge_types

{'regular': 2, 'guest': 47, 'interim': 1}

### Web Scraping (15 Minutes)
Scrape the website [http://quotes.toscrape.com/](http://quotes.toscrape.com/).  Turn each quote on the page into a Pandas dataframe consisting of two columns: the author of the quote and the quote itself. The final result should be a dataframe of 10 rows (not counting header) and 2 columns (not counting index).

In [56]:
resp = requests.get('http://quotes.toscrape.com/')
soup = BeautifulSoup(resp.content, 'html.parser')

In [64]:
boxes = soup.findAll('div', class_='quote')

In [69]:
quotes = [box.find('span', class_='text').text for box in boxes]
print(len(quotes))
quotes

10


['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

In [70]:
authors = [box.find('small').text for box in boxes]
print(len(authors))
authors

10


['Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Jane Austen',
 'Marilyn Monroe',
 'Albert Einstein',
 'André Gide',
 'Thomas A. Edison',
 'Eleanor Roosevelt',
 'Steve Martin']

In [74]:
df = pd.DataFrame([authors, quotes])
df = df.T
df.columns = ['Authors', 'Quotes']
df

,Authors,Quotes
0,Albert Einstein,“The world as we have created it is a process ...
1,J.K. Rowling,"“It is our choices, Harry, that show what we t..."
2,Albert Einstein,“There are only two ways to live your life. On...
3,Jane Austen,"“The person, be it gentleman or lady, who has ..."
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and..."
5,Albert Einstein,“Try not to become a man of success. Rather be...
6,André Gide,“It is better to be hated for what you are tha...
7,Thomas A. Edison,"“I have not failed. I've just found 10,000 way..."
8,Eleanor Roosevelt,“A woman is like a tea bag; you never know how...
9,Steve Martin,"“A day without sunshine is like, you know, nig..."


### NoSQL (10 Minutes)
Answer all of the following questions by querying Mongo and manipulating the results in Python
1. Load data from `assets/grades.json` into Mongo (this code is written for you)

In [75]:
# you shouldn't need to edit this cell!
with open('assets/grades.jsonl') as f:
    # loads() comes from the bson library
    file_data = [loads(line) for line in f.readlines()]

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mod2db"]
coll = db["testcoll"]

coll.insert_many(file_data)

2. How many records are there total?

In [77]:
num_records = coll.count_documents({})
num_records

280

3. How many students have taken the class with `class_id` = **29**?

In [81]:
coll.find({})[0]

{'_id': ObjectId('50b59cd75bed76f46522c34e'),
 'student_id': 0,
 'class_id': 2,
 'scores': [{'type': 'exam', 'score': 57.92947112575566},
  {'type': 'quiz', 'score': 21.24542588206755},
  {'type': 'homework', 'score': 68.1956781058743},
  {'type': 'homework', 'score': 67.95019716560351},
  {'type': 'homework', 'score': 18.81037253352722}]}

In [90]:
step1 = coll.find({'class_id':29}, {'_id':0,'student_id':1})
students = []
for student in step1:
    students.append(student['student_id'])
len(set(students))

9

4. Super bonus question: for student **12** in class **23**, what grade did they get on their exam?

In [98]:
step1 = coll.find({'class_id':23, 'student_id':12}, {'_id':0,'scores':1})
for x in step1:
    for result in x['scores']:
        if result['type'] == 'exam':
            print(result['score'])

26.9857216299485


In [99]:
# for when you're done with this portion, this deletes the data we added.
client.drop_database("mod2db")

## Assessment submission (2 Minutes)
Please save your completed file as `mod2-assessment.ipynb` and upload it using [this form](https://docs.google.com/forms/d/e/1FAIpQLSczNVM9Q6Dr3w1YR3kkg4PNaOsfDO-qIvh1wLy6rR0SEwTwIQ/viewform?usp=sf_link)

## Clifford Bridges